# COMP47590: Advanced Machine Learning
# Assignment 1: Implementing Perceptrons

- Student 1 Name: Carl Fabian Winkler
- Student 1 Number: 20207528
- Student 2 Name: David Moreno Boras
- Student 2 Number: 21200646

## Import Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split

"""
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.utils.validation import check_X_y, check_array, check_is_fitted, check_random_state
from sklearn.utils.multiclass import unique_labels
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.utils import resample"""

'\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.model_selection import cross_val_score\nfrom sklearn.model_selection import train_test_split\n\nfrom sklearn.utils.validation import check_X_y, check_array, check_is_fitted, check_random_state\nfrom sklearn.utils.multiclass import unique_labels\nfrom sklearn import preprocessing\nfrom sklearn import metrics\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.model_selection import cross_val_score\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.utils import resample'

## Task 1: The Perceptron Classifier

Define the PerceptronClassifier class

In [21]:
class Layer():
    def __init__(self, n_in, n_out, activation = 'Sigmoid', init = 'Xavier'):
        self.activation = activation
        # XW + b = y ; We input more than one sample per pass...
        
        self.weights = np.zeros((n_in, n_out))
        self.biases = np.zeros((n_out))
        if init == 'Xavier':
            var = np.sqrt(6.0 / (n_in + n_out))
            for i in range(n_in):
                for j in range(n_out):
                      self.weights[i,j] = np.float32(np.random.uniform(-var, var))
        print("Weights:", self.weights.shape)
        print(self.weights) 
        
    def getWeights(self):
        return self.weights

    def forward(self, x):
        print("X:", x.shape)
        print(x)
        print("Weights:", self.weights.shape)
        print(self.weights)
        z = x @ self.weights + self.biases
        
        if self.activation == 'Sigmoid':
            out = 1 / (1 + np.exp(-z))
        elif self.activation == 'ReLu':
            out = np.maximum(z, 0)
        elif self.activation == 'TanH':
            out = np.tanh(z)
        else:
            out = z
        
        self.cache = (x, z)
        
        return out    
    
    def backward(self, d_out):
        inputs, z = self.cache
        weight = self.weights
        bias = self.biases
        
        if self.activation == 'Sigmoid':
            d_act = d_out * (1 / (1 + np.exp(-z))) * (1 - 1 / (1 + np.exp(-z)))
        elif self.activation == 'ReLu':
            d_act = d_out * (z > 0)
            
        elif self.activation == 'TanH':
            d_act = d_out * np.square(z)
        else:
            d_act = z
            
        d_inputs = d_act @ weight.T
        d_weight = inputs.T @ d_act
        d_bias = d_act.sum(axis=0) 
        
        self.d_w = d_weight
        self.d_b = d_bias
        
        return d_inputs, d_weight, d_bias
    
    def update_gd_params(self, lr):
        self.weights = self.weights - lr * self.d_w
        self.biases = self.biases - lr * self.d_b

class PerceptronClassifier(BaseEstimator, ClassifierMixin, ):
    
    """
    Parameters
    ----------
    Attributes
    ----------
    Notes
    -----
    See also
    --------
    Examples
    --------
    """
    # Constructor for the classifier object
    def __init__(self, in_dim, out_dim, hidden_units, n_layers, activation = 'Sigmoid', 
                 learning_rate = 0.01, weight_decay = 0, epochs = -1, regularisation = 'L2'):

        """Setup a Perceptron classifier .
        Parameters
        ----------
        Returns
        -------
        """     
        
        self.layers = []
        self.lr = learning_rate
        self.regularisation = regularisation
        
        self.layers.append(Layer(in_dim, hidden_units, activation, 'Xavier'))
        for l in range(n_layers):
            self.layers.append(Layer(hidden_units, hidden_units, activation, 'Xavier'))
        self.layers.append(Layer(hidden_units, out_dim, activation, 'Xavier'))
        
        print("Layers:", len(self.layers))
        
        # Initialise class variabels
    def forward(self, X):
        out = self.layers[0].forward(X)
        for layer in self.layers[1:]:
            out = layer.forward(out)
        return out
                
    def backward(self, in_grad):
        i = len(self.layers) - 2 
        # d_inputs, _, _ = lay.backward(in_grad)
        next_grad = self.layers[i+1].backward(in_grad)
        while i >= 1:
            next_grad = self.layers[i].backward(next_grad)
            i -= 1
        
    def l2_loss(self, y_hat, pred):
        # totalSum = 0
        # for layer in self.layers:
        #     totalSum = totalSum + np.sum(np.sum(layer.getWeights())
        return -y_hat-pred

    def loss(self, y_hat, pred):
        return -y_hat-pred
        
    # The fit function to train a classifier
    def fit(self, X, y, epochs = 30):
        # WRITE CODE HERE
        for i in range(epochs):
            out = self.forward(X)
            if (self.regularisation == 'L2'):
                grad = self.l2_loss(y, out)
            else:
                grad = self.loss(y, out)
                
            # Backpropagation
            self.backward(grad)
            
            # Update weights and biases
            for layer in self.layers:
                layer.update_gd_params(self.lr)
        return
    
    # The predict function to make a set of predictions for a set of query instances
    def predict(self, X):
        return self.forward(X)
    
    # The predict_proba function to make a set of predictions for a set of query instances. This returns a set of class distributions.
    def predict_proba(self, X):
        tmp = self.forward(X)
        sum1 = tmp.sum(axis = 1)
        out = X.T / sum1
        out = out.T
        return out

## Task 2: Evaluation

Load the Diabethic Retinopathy dataset

In [ ]:
x = np.array([[1,1],[2,2]])


D = 2
N = 2
H = 1


weights = np.ones((2,1))
biases = np.ones((1))  
              


In [ ]:
x = np.array([[1,1],[2,1]])

In [ ]:
import numpy as np
x = np.array([[1,1,2],[2,1,3]])
sum1 = x.sum(axis = 1)
x = x.T / sum1
x = x.T

In [22]:
x = np.array([[1,1],[2,2]])
y = np.ones(1)
clf = PerceptronClassifier(2, 1, 1, 1, regularisation='None')

Weights: (2, 1)
[[-0.72654527]
 [ 0.4850854 ]]
Weights: (1, 1)
[[1.43463349]]
Weights: (1, 1)
[[0.60716546]]
Layers: 3


In [23]:
clf.fit(x, y)
# clf.predict(x)

X: (2, 2)
[[1 1]
 [2 2]]
Weights: (2, 1)
[[-0.72654527]
 [ 0.4850854 ]]
X: (2, 1)
[[0.43992662]
 [0.38156291]]
Weights: (1, 1)
[[1.43463349]]
X: (2, 1)
[[0.65274643]
 [0.63353284]]
Weights: (1, 1)
[[0.60716546]]


/tmp/ipykernel_47510/533334978.py:45: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  d_act = d_out * (1 / (1 + np.exp(-z))) * (1 - 1 / (1 + np.exp(-z)))


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1 is different from 3)

Evaluate the perfomrance of the perceptron classifier on the daibetic retinopathy dataset.

In [ ]:
plt.plot(outpur)

## Task 3 & 4: Add Different Activations & Regularisation

Reimplement the PerceptronClassifier class adding an activation function option and L2 regularisation. 

In [ ]:
class PerceptronClassifier2(BaseEstimator, ClassifierMixin):
    """
    """
    # Constructor for the classifier object
    def __init__(self):

        """Setup a Perceptron classifier .
        Parameters
        ----------

        
        Returns
        -------

        """     

        # Initialise ranomd state if set
        self.random_state = random_state
        
        # Initialise class variabels

        
    # The fit function to train a classifier
    def fit(self, X, y):
        
        # WRITE CODE HERE
        

    # The predict function to make a set of predictions for a set of query instances
    def predict(self, X):

        # WRITE CODE HERE
    
    # The predict_proba function to make a set of predictions for a set of query instances. This returns a set of class distributions.
    def predict_proba(self, X):
        
        # WRITE CODE HERE

Load the dataset and explore it.

## Task 5: Reflect on the Performance of the Different Models Evaluated

Perform hyper-parameter tuning and evaluate models. 

## Test the Diabetic Retiniphaty dataset

In [ ]:
diabetic_af = pd.read_csv('messidor_features.csv', na_values='?', index_col = 0)
diabetic_af.head()
y = diabetic_af.pop('Class').values
x_raw = diabetic_af.values
print("Features: ", x_raw[0:2])
print("Class: ", y[0:10])

### Train and predict using our classifier

With a single split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_raw, y, shuffle=True, train_size = 0.7)
clf = PerceptronClassifier(len(x_train[0]), 1, )
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
accuracy = metrics.accuracy_score(y_pred, y_test)

### Grid search

Do grid search with the train set, use the test set for evaluation

In [ ]:
cv_folds = 5
param_grid ={'activation': ['Sigmoid', 'ReLu', 'TanH'], 'regularisation':['None', 'L2']}

# Perform the search
tuned_perceptron = GridSearchCV(PerceptronClassifier(), \
                            param_grid, cv=cv_folds, verbose = 2, \
                            n_jobs = -1)
cross_val_score(clf, x_train, y_train, cv=10)